# Gradient-based optimization using automatic differentiation

In this notebook, we optimize the minor radius and elongation of an axisymmetric torus to obtain a desired volume and area. The problem thus has two degrees of freedom. The objective function has least-squares form with two terms, involving the surface area and enclosed volume. VMEC is not used. This example is equivalent to `IntegratedTests.test_2dof_surface_opt` in `simsopt/tests/test_integrated.py`.

In [1]:
import numpy as np
from simsopt.geo.surfacerzfourier import SurfaceRZFourier
from simsopt.objectives.least_squares import LeastSquaresProblem
from simsopt.solve.serial import least_squares_serial_solve

Specify the values we'd like to achieve:

In [2]:
desired_volume = 0.6
desired_area = 8.0

Start with a default toroidal surface, which is axisymmetric with major radius 1 and minor radius 0.1.

In [3]:
surf = SurfaceRZFourier()

We are free to modify the surface shape, like this:

In [4]:
surf.set_zs(1, 0, 0.2)

or like this:

In [5]:
surf.set('rc(1,0)', 0.09)

The surface Fourier modes are all non-fixed by default, meaning they will be optimized.  You can choose to exclude any subset of the variables from the space of independent variables by setting their `fixed` property to `True`:

In [6]:
surf.set_fixed('rc(0,0)')

Each function we wish to optimize is then equipped with a shift and weight, to become a term in a least-squares objective function. The form of each term is $weight(function - goal)^2$.

In [7]:
term1 = (surf.volume, desired_volume, 1)
term2 = (surf.area,   desired_area,   1)

A list of terms are combined to form a nonlinear-least-squares problem:

In [8]:
prob = LeastSquaresProblem([term1, term2])

Let's print out the initial global state vector, i.e. the vector of variables that is optimized. Each entry in this state vector has an associated string, explaining its meaning.

In [9]:
print(prob.x)
print(prob.dofs.names)

[0.09 0.2 ]
['rc(1,0) of SurfaceRZFourier 0x125de0f40 (nfp=1, stellsym=True, mpol=1, ntor=0)', 'zs(1,0) of SurfaceRZFourier 0x125de0f40 (nfp=1, stellsym=True, mpol=1, ntor=0)']


We asked to optimize the functions `area` and `volume` of `surf`. Since `surf` also has functions with names that are the same except for a `d` in front (i.e. `darea` and `dvolume`), simsopt detects that derivative information is available.

In [10]:
dir(surf)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_validate_mn',
 'all_fixed',
 'allocate',
 'area',
 'change_resolution',
 'darea',
 'darea_by_dcoeff',
 'dependencies',
 'dgamma_by_dcoeff',
 'dgammadash1_by_dcoeff',
 'dgammadash2_by_dcoeff',
 'dnormal_by_dcoeff',
 'dvolume',
 'dvolume_by_dcoeff',
 'extend_via_normal',
 'fit_to_curve',
 'fixed',
 'fixed_range',
 'from_focus',
 'gamma',
 'gammadash1',
 'gammadash2',
 'get',
 'get_dofs',
 'get_fixed',
 'get_rc',
 'get_rs',
 'get_zc',
 'get_zs',
 'index',
 'invalidate_cache',
 'least_squares_fit',
 'make_names',
 'make_names_helper',
 'mpol',
 'names',
 'nfp',
 'normal',
 'ntor',
 'plot',
 'quadpoints_phi',
 'quadpoints_th

In [11]:
prob.dofs.grad_avail

True

Finally, let's solve the optimization problem. Since simsopt has detected that analytic derivatives are available, it chooses a derivative-based algorithm.

In [12]:
least_squares_serial_solve(prob)

Using derivatives
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.1679e+00                                    4.76e+01    
       1              2         8.8652e-04      2.17e+00       7.67e-02       2.26e-01    
       2              3         1.6220e-05      8.70e-04       1.34e-02       1.23e-01    
       3              4         1.3232e-10      1.62e-05       7.57e-04       3.41e-04    
       4              5         1.4763e-20      1.32e-10       2.43e-06       3.62e-09    
`gtol` termination condition is satisfied.
Function evaluations 5, initial cost 2.1679e+00, final cost 1.4763e-20, first-order optimality 3.62e-09.


Let's examine the optimum:

In [13]:
print("At the optimum,")
print(" rc(m=1,n=0) = ", surf.get_rc(1, 0))
print(" zs(m=1,n=0) = ", surf.get_zs(1, 0))
print(" volume = ", surf.volume())
print(" area = ", surf.area())
print(" objective function = ", prob.objective())

At the optimum,
 rc(m=1,n=0) =  0.10962565114183004
 zs(m=1,n=0) =  0.2772741121549299
 volume =  0.5999999999418775
 area =  8.0000000001617
 objective function =  2.952511150728531e-20


The optimization achieved the desired volume and area.